# Building a Stock Analysis Application

*[Coding along with the Udemy online course AI Agents: Building Teams of LLM Agents that Work For You by Mohsen Hassan & Ilyass Tabiai]*

__The notebook here has the mere purpose of describing and explaining the code we're going to use, not of actually run the code.__

In [1]:
import pandas as pd
api_key = pd.read_csv("~/tmp/chat_gpt/autogen_agent_1.txt", sep=" ", header=None)[0][0]
llm_config = {
    "model": "gpt-4o",
    "api_key": api_key
    }
print("Don't be a fool and send your api key to GitHub!")

Don't be a fool and send your api key to GitHub!


## Application: Financial Report

We are now going to write a complex streamlit app that will leverage agents to accomplish a specific task. We will later deploy this app on Google Cloud so that it gets a permanent link you can share with others to let them test your work.

In [2]:
import autogen

## Goals of our App

For this final example, we are going to create an app that re-uses some of the features we previously explored together. This app will do the following *<span style="color:blue">(description and goals are from the Udemy course [AI Agents: Building Teams of LLM Agents that Work For You ](https://www.udemy.com/course/ai-agents-building-teams-of-llm-agents-that-work-for-you/) by Mohsen Hassan and Ilyass Tabiai, PhD.)</span>*:

* Takes as an input asset tickers
* Downloads asset price data, analyzes them (correlations, returns, ...)
* Retrieve or compute the following ratios:
    * P/E ratio
    * Forward P/E
    * Dividends
    * Price to book
    * Debt/Eq
    * ROE
* Analyze the correlation between the stocks
* Plot their normalized prices for comparison
* Downloads the latest news headlines about these assets and then analyzes and summarizes these news headlines
* Writes a financial report about these assets based on all this information
    * **Legal reviewer**: Check that the content is legally compliant (in case we share it with others)
    * **Text/Data alignment reviewer**: Check that the text content is aligned with the text written in the report
    * **Consistency reviewer**: Check that the conclusion are consistent throughout the text and no contradictions are present
    * **Completion reviewer**: Check that the report contains all the elements requested
* Refines the report based on previous criticisms
* Saves the final report to a mardown file with a normalized price chart


## Defining our Agents

Our app will rely on agents to accomplish its goal, in this section, we will present the structure it will follow.

### 1. Financial & Research Task

The first task will be to get the financial data about the chosen assets and compute some of their performance metrics, the second task will be to investigate reasons for this performance based on news headlines. Let's first provide a prompt for these tasks. Note that this time we will provide both of these prompts in a list for convenience:

In [3]:
from datetime import datetime
date_str = datetime.now().strftime("%Y-%m-%d")

stocks = "AMD, NVDA"

financial_tasks = [
    f"""Today is the {date_str}. 
    What are the current stock prices of {stocks}, and how is the performance over the past 6 months in terms of percentage change? 
    Start by retrieving the full name of each stock and use it for all future requests.
    Prepare a figure of the normalized price of these stocks and save it to a file named normalized_prices.png. Include information about, if applicable: 
    * P/E ratio
    * Forward P/E
    * Dividends
    * Price to book
    * Debt/Eq
    * ROE
    * Analyze the correlation between the stocks
    Do not use a solution that requires an API key.
    If some of the data does not makes sense, such as a price of 0, change the query and re-try.""",

    """Investigate possible reasons of the stock performance leveraging market news headlines from Bing News or Google Search. Retrieve news headlines using python and return them. Use the full name stocks to retrieve headlines. Retrieve at least 10 headlines per stock. Do not use a solution that requires an API key.""",
]

In [4]:
financial_tasks # important to note: Do not use a solution that requires an API key

['Today is the 2024-10-09. \n    What are the current stock prices of AMD, NVDA, and how is the performance over the past 6 months in terms of percentage change? \n    Start by retrieving the full name of each stock and use it for all future requests.\n    Prepare a figure of the normalized price of these stocks and save it to a file named normalized_prices.png. Include information about, if applicable: \n    * P/E ratio\n    * Forward P/E\n    * Dividends\n    * Price to book\n    * Debt/Eq\n    * ROE\n    * Analyze the correlation between the stocks\n    Do not use a solution that requires an API key.\n    If some of the data does not makes sense, such as a price of 0, change the query and re-try.',
 'Investigate possible reasons of the stock performance leveraging market news headlines from Bing News or Google Search. Retrieve news headlines using python and return them. Use the full name stocks to retrieve headlines. Retrieve at least 10 headlines per stock. Do not use a solution t

We now need agents to perform these taks. We are going to define two agents to do so, a Financial agent, for the first task, and a Research agent, for the second task. Both agents will have to provide code that can perform each task, so these will be Assistant agents with LLM capability:

In [5]:
financial_assistant = autogen.AssistantAgent(
    name="Financial_assistant",
    llm_config=llm_config,
)
research_assistant = autogen.AssistantAgent(
    name="Researcher",
    llm_config=llm_config,
)

[autogen.oai.client: 10-09 14:37:23] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
[autogen.oai.client: 10-09 14:37:23] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


At the end of this task, we will have the price data, performance metrics and a chart that shows normalized price evolution fot the specified assets. 

### 2. Writing task

The second taks will be to write a first version of a report based on the data provided by the previous agents. Let's define the task:

In [6]:
writing_tasks = [
        """Develop an engaging financial report using all information provided, include the normalized_prices.png figure,
        and other figures if provided.
        Mainly rely on the information provided. 
        Create a table comparing all the fundamental ratios and data.
        Provide comments and description of all the fundamental ratios and data.
        Compare the stocks, consider their correlation and risks, provide a comparative analysis of the stocks.
        Provide a summary of the recent news about each stock. 
        Ensure that you comment and summarize the news headlines for each stock, provide a comprehensive analysis of the news.
        Provide connections between the news headlines provided and the fundamental ratios.
        Provide an analysis of possible future scenarios. 
        """]

In [7]:
writing_tasks

['Develop an engaging financial report using all information provided, include the normalized_prices.png figure,\n        and other figures if provided.\n        Mainly rely on the information provided. \n        Create a table comparing all the fundamental ratios and data.\n        Provide comments and description of all the fundamental ratios and data.\n        Compare the stocks, consider their correlation and risks, provide a comparative analysis of the stocks.\n        Provide a summary of the recent news about each stock. \n        Ensure that you comment and summarize the news headlines for each stock, provide a comprehensive analysis of the news.\n        Provide connections between the news headlines provided and the fundamental ratios.\n        Provide an analysis of possible future scenarios. \n        ']

In [8]:
# let's define an agent to work on the writing task
writer = autogen.AssistantAgent(
    name="writer",
    llm_config=llm_config,
    system_message="""
        You are a professional writer, known for
        your insightful and engaging finance reports.
        You transform complex concepts into compelling narratives. 
        Include all metrics provided to you as context in your analysis.
        Only answer with the financial report written in markdown directly, do not include a markdown language block indicator.
        Only return your final work without additional comments.
        """,
)
writer

[autogen.oai.client: 10-09 14:37:31] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


### 3. Refining the blogpost

To refine the blogpost based on SEO, legal aspects and ethical aspects, we are going to use __a nested chat that will be triggered when the Writer contacts the Critic__. The Critic will trigger __a nested chat with agents designed to optimize each aspect__. A final agent in this nested chat, the Meta Reviewer, will summarize and assemble all reviews and send them back to the Critic, who'll transfer them to the Writer agent to prepare a refined version. Let's define all these agents:

In [9]:
critic = autogen.AssistantAgent(
    name="Critic",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="You are a critic. You review the work of "
                "the writer and provide constructive "
                "feedback to help improve the quality of the content.",
)

legal_reviewer = autogen.AssistantAgent(
    name="Legal_Reviewer",
    llm_config=llm_config,
    system_message="You are a legal reviewer, known for "
        "your ability to ensure that content is legally compliant "
        "and free from any potential legal issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

consistency_reviewer = autogen.AssistantAgent(
    name="Consistency_reviewer",
    llm_config=llm_config,
    system_message="You are a consistency reviewer, known for "
        "your ability to ensure that the written content is consistent throughout the report. "
        "Refer numbers and data in the report to determine which version should be chosen " 
        "in case of contradictions. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",
)

textalignment_reviewer = autogen.AssistantAgent(
    name="Text_alignment_reviewer",
    llm_config=llm_config,
    system_message="You are a text data alignment reviewer, known for "
        "your ability to ensure that the meaning of the written content is aligned "
        "with the numbers written in the text. " 
        "You must ensure that the text clearely describes the numbers provided in the text "
        "without contradictions. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",
)

completion_reviewer = autogen.AssistantAgent(
    name="Completion_Reviewer",
    llm_config=llm_config,
    system_message="You are a content completion reviewer, known for "
        "your ability to check that financial reports contain all the required elements. "
        "You always verify that the report contains: a news report about each asset, " 
        "a description of the different ratios and prices, "
        "a description of possible future scenarios, a table comparing fundamental ratios and "
        " at least a single figure. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",
)

meta_reviewer = autogen.AssistantAgent(
    name="Meta_Reviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggregate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)

[autogen.oai.client: 10-09 14:37:35] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
[autogen.oai.client: 10-09 14:37:35] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
[autogen.oai.client: 10-09 14:37:35] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
[autogen.oai.client: 10-09 14:37:35] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
[autogen.oai.client: 10-09 14:37:35] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
[autogen.oai.client: 10-09 14:37:35] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


### 4. Exporting the blogpost in markdown

Finally, we will define one last agent that will write the python code necessary to export the final blogpost in a makrdown file:

In [10]:
exporting_task = ["""Save the blogpost and only the blogpost to a .md file using a python script."""]

In [11]:
export_assistant = autogen.AssistantAgent(
    name="Exporter",
    llm_config=llm_config,
)

[autogen.oai.client: 10-09 14:37:44] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


### Code Execution Agent

Finally, we will need an __agent to execute code__. Since we're preparing an app that will have to run without human intervention, we will create a __user proxy agent that will never request human input__. This is a bit risky as the agents might get stuck in an execution loop of code that does not work; they might also execute malicious code (the risk is very low for this app). Let's define such an agent:

In [12]:
user_proxy_auto = autogen.UserProxyAgent(
    name="User_Proxy_Auto",
    human_input_mode="NEVER",
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    code_execution_config={
        "last_n_messages": 3,
        "work_dir": "coding",
        "use_docker": False,
    },  
)

## Agentic Flow: Orchestrating the Chat

We are now going to define the flow of chats through which our agents will interact. We will have to define two chats:
1. The nested chat flow for blog post writing refining
2. The main chat flow

### 1. Nested Chat Flow

Let's first start by defining the nested chat flow. The chat structure will be very similar to the one defined in one of the previous apps we worked on.

This chat will be triggered when the Writer will contact the Critic:

In [13]:
def reflection_message(recipient, messages, sender, config):
    return f'''Review the following content. 
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

In [14]:
review_chats = [
    {
    "recipient": legal_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into a JSON object only:"
        "{'Reviewer': '', 'Review': ''}.",},
     "max_turns": 1},
    {"recipient": textalignment_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into a JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},
    {"recipient": consistency_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into a JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},
    {"recipient": completion_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into a JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},
     {"recipient": meta_reviewer, 
      "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.", 
     "max_turns": 1},
]

In [15]:
critic.register_nested_chats(
    review_chats,
    trigger=writer,
)

### Main Chat Flow

__Let's now define the main chat flow, it will be composed of 4 chats:__

1. __Financial agent__ with user proxy to accomplish the first financial task
2. __Research agent__ with user proxy to accomplish the second financial task
3. __Critic with Writer__ to write and refine the blogpost through the Nested chat
4. __Export agent__ with user proxy to export the markdown blogpost to a file

In [16]:
# code that would execute our task but it's not supposed to run inside a notebook :)
# or is it?
chat_results = autogen.initiate_chats(
    [
        {
            "sender": user_proxy_auto,
            "recipient": financial_assistant,
            "message": financial_tasks[0],
            "silent": False,
            "summary_method": "reflection_with_llm",
            "summary_args": {
                "summary_prompt" : "Return the stock prices of the stocks, their performance and all other metrics"
                "into a JSON object only. Provide the name of all figure files created. Provide the full name of each stock.",
                            },
            "clear_history": False,
            "carryover": "Wait for confirmation of code execution before terminating the conversation. Verify that the data is not completely composed of NaN values. Reply TERMINATE in the end when everything is done."
        },
        {
            "sender": user_proxy_auto,
            "recipient": research_assistant,
            "message": financial_tasks[1],
            "silent": False,
            "summary_method": "reflection_with_llm",
            "summary_args": {
                "summary_prompt" : "Provide the news headlines as a paragraph for each stock, be precise but do not consider news events that are vague, return the result as a JSON object only.",
                            },
            "clear_history": False,
            "carryover": "Wait for confirmation of code execution before terminating the conversation. Reply TERMINATE in the end when everything is done."
        },
        {
            "sender": critic,
            "recipient": writer,
            "message": writing_tasks[0],
            "carryover": "I want to include a figure and a table of the provided data in the financial report.",
            "max_turns": 2,
            "summary_method": "last_msg",
        },
        {
            "sender": user_proxy_auto,
            "recipient": export_assistant,
            "message": exporting_task[0],
            "carryover": "Wait for confirmation of code execution before terminating the conversation. Reply TERMINATE in the end when everything is done.",
        }
    ]
)


********************************************************************************
Starting a new chat....

********************************************************************************
User_Proxy_Auto (to Financial_assistant):

Today is the 2024-10-09. 
    What are the current stock prices of AMD, NVDA, and how is the performance over the past 6 months in terms of percentage change? 
    Start by retrieving the full name of each stock and use it for all future requests.
    Prepare a figure of the normalized price of these stocks and save it to a file named normalized_prices.png. Include information about, if applicable: 
    * P/E ratio
    * Forward P/E
    * Dividends
    * Price to book
    * Debt/Eq
    * ROE
    * Analyze the correlation between the stocks
    Do not use a solution that requires an API key.
    If some of the data does not makes sense, such as a price of 0, change the query and re-try.
Context: 
Wait for confirmation of code execution before terminating the c

/Users/juergenkober/Projects/Python+Rust/_github-repos/working-with-financial-data/py-building-llm-agent-teams/env/lib/python3.12/site-packages/autogen/agentchat/conversable_agent.py:1264: UserWarning: Cannot extract summary using reflection_with_llm: Error code: 400 - {'error': {'message': "Invalid 'messages[1].name': string does not match pattern. Expected a string that matches the pattern '^[a-zA-Z0-9_-]+$'.", 'type': 'invalid_request_error', 'param': 'messages[1].name', 'code': 'invalid_value'}}. Using an empty str as summary.
  warnings.warn(


Text alignment reviewer (to Critic):

**Role: Text Data Alignment Reviewer**

- In the P/E and Forward P/E discussion, it's stated that AMD has a high P/E indicating a premium, while NVDA has a relatively moderate P/E ratio despite it being lower than AMD's. The text suggests the Forward P/E for AMD is lower, yet NVDA’s Forward P/E is actually slightly higher (33.06 vs 31.82). Clarify whether "moderate" is in comparison to the industry or AMD's other figures.

- On Debt/Equity ratios, the text states both companies hold high ratios with NVDA's substantially higher. The table shows AMD's is actually lower (3.97) compared to NVDA's (17.22). Adjust the commentary to accurately reflect that NVDA's use of leverage is more significant. 

- The correlation analysis notes a divergence potential between these stocks, but states AMD's correlation with itself as 1.0, which is misleading. Clarify the apparent autocorrelation within AMD and discuss NVDA's correlation with AMD (0.2724) as a diversif